# Lab02: Web Crawler (Continue).

- MSSV: 18120061
- Họ và tên: Lê Nhựt Nam

## Yêu cầu bài tập

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; trong file, từ `TODO` để cho biết những phần mà bạn cần phải làm.

Bạn có thể thảo luận ý tưởng cũng như tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn*. 

Nếu vi phạm thì sẽ bị 0 điểm cho bài tập này.

**Cách nộp bài**

Trước khi nộp bài, rerun lại notebook (`Kernel` -> `Restart & Run All`).

Bạn nộp file notebook với tên `MSSV.ipynb` của bạn (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên file là `1234567.ipynb`) và nộp file này trên moodle.

**Nội dung bài tập**

Thu thập và thể hiện dữ liệu web.

## 2. Cài đặt

### 2.1. Import library

In [1]:
#!pip install beautifulsoup4
# If you use Anaconda env or Minianconda
#!conda install -c anaconda beautifulsoup4

import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string

### 2.2. HTML Parser


Bộ phân tích cú pháp HTML (HTML Parser): nhận HTML code và trích xuất thông tin liên quan như tiêu đề của trang, đoạn văn trong trang, tiêu đề trong trang, liên kết, văn bản in đậm, v.v.

In [2]:
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Start tag:", tag)

    def handle_endtag(self, tag):
        print("End tag :", tag)

    def handle_data(self, data):
        print("Data  :", data)

parser = MyHTMLParser()
parser.feed('<html><head><title>Test</title></head>')

Start tag: html
Start tag: head
Start tag: title
Data  : Test
End tag : title
End tag : head


#### Loại bỏ các HTML tag không cần thiết bằng cách thiết lập filter

In [3]:
def text_filter(element):
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True

### 2.3 Thu thập nội dung trang Web

#### Trong bài tập này ta thể hiện tài liệu (hay nội dung trang web) với cấu trúc đơn giản như sau: 
- Gọi $D$ là một tập tài liệu chứa *n* tài liệu: $D=\left\{d_1,d_2,...,d_n\right\}$.
- Ta thể hiện tài liệu bằng một dictionary `data={}` với `data[word]=[[url_idx_1,url_idx_2,...],frequency]` với `url_index`$\in{\left[{1,n}\right]}$

VD: `data['at']=[[1,2], 5]`
Từ `at` xuất hiện trong đường dẫn có index `1` và `2` tổng số lần xuất hiện là 5.

#### Bước 1: liệt kê các từ xuất hiện trong trang web:

In [4]:
def wordList(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    text = soup.findAll(text=True)
    filtered_text = list(filter(text_filter, text)) # list của các chuỗi
    word_list = []
    # TODO:
    # Với mỗi chuỗi trong filtered_text:
    for text in filtered_text:
        #   Thay thế các dấu câu thành khoảng trắng (gợi ý: danh sách các dấu câu: string.punctuation; thay thế: .replace(...))
        #   Sử dụng .join() function, nếu một char c nào đó nằm trong string.punctuation thì nó sẽ được thay thế bằng ' '. Ngược lại thì không thay đổi gì
        # sau đó join với một string rỗng để có được clean_text
        clean_text = ''.join(' ' if c in string.punctuation else c for c in text)
        #   Tách chuỗi bởi khoảng trắng (.split(...))
        # cắt clean_text vừa có ở trên ra
        lst_words_in_clean_text = clean_text.split()
        #   Thêm các từ vừa được tách ra vào word_list
        word_list = word_list + lst_words_in_clean_text
    return word_list
  
# Test
print(wordList('https://en.wikipedia.org/wiki/Web_mining'))

['Web', 'mining', 'From', 'Wikipedia', 'the', 'free', 'encyclopedia', 'This', 'article', 'includes', 'a', 'related', 'reading', 'or', 'but', 'its', 'sources', 'remain', 'unclear', 'because', 'it', 'lacks', 'October', '2020', 'This', 'article', 'may', 'require', 'The', 'specific', 'problem', 'is', 'The', 'article', 'needs', 'sufficient', 'inline', 'references', 'and', 'a', 'complete', 're', 'write', 'has', 'been', 'proposed', 'October', '2020', 'Web', 'mining', 'It', 'uses', 'automated', 'methods', 'to', 'extract', 'both', 'structured', 'and', 'unstructured', 'data', 'from', 'web', 'pages', 'server', 'logs', 'and', 'link', 'structures', 'There', 'are', 'three', 'main', 'sub', 'categories', 'of', 'web', 'mining', 'Contents', '1', 'Web', 'mining', 'types', '2', 'Web', 'usage', 'mining', '2', '1', 'Pros', '2', '2', 'Cons', '3', 'Web', 'structure', 'mining', '4', 'Web', 'content', 'mining', '4', '1', 'Web', 'content', 'mining', 'in', 'foreign', 'languages', '4', '1', '1', 'Chinese', '5', 'S

#### Bước 2: Tính tần suất xuất hiện của từ trong 1 trang web, lưu trữ dữ liệu vào data:

In [5]:
def read_url(url, url_idx, data):
    word_list = wordList(url)
    # TODO
    # Với mỗi từ w trong word_list:
    for i, word in enumerate(word_list):
        #   Nếu w chưa có trong data thì khởi tạo data[w] = [[url_idx], 1]
        if word not in data.keys():
            data[word]=[[url_idx],1]
        #   Ngược lại thì thêm url_idx vào data[w][0] (nếu chưa có) và tăng data[w][1] lên 1 đơn vị
        else:
            if url_idx in data[word][0]:
                data[word][0].append(i)
            data[word][1]+=1

# Test
test_data = {}
read_url('https://en.wikipedia.org/wiki/Web_mining', 1, test_data)
test_data

{'Web': [[1,
   48,
   80,
   84,
   94,
   98,
   103,
   125,
   128,
   138,
   141,
   144,
   154,
   162,
   165,
   168,
   184,
   187,
   202,
   211,
   213,
   229,
   243,
   318,
   512,
   533,
   537,
   552,
   555,
   571,
   631,
   693,
   710,
   766,
   787,
   793,
   810,
   836,
   878,
   895],
  40],
 'mining': [[1,
   49,
   77,
   81,
   86,
   96,
   100,
   105,
   126,
   129,
   140,
   143,
   146,
   155,
   160,
   164,
   167,
   170,
   186,
   189,
   215,
   245,
   285,
   303,
   320,
   453,
   487,
   514,
   535,
   539,
   549,
   554,
   557,
   560,
   621,
   633,
   694,
   712,
   739],
  39],
 'From': [[1], 1],
 'Wikipedia': [[1, 963], 2],
 'the': [[1,
   151,
   191,
   197,
   223,
   306,
   316,
   352,
   364,
   367,
   375,
   385,
   414,
   423,
   435,
   460,
   463,
   468,
   484,
   508,
   551,
   559,
   577,
   652,
   660,
   663,
   669,
   687,
   697,
   703,
   722,
   726,
   918],
  33],
 'free': [[1], 1],
 'enc

#### Bước 3: Chạy chương trình lưu kết quả vào file output.txt

In [6]:
data = {}
read_url('https://en.wikipedia.org/wiki/Web_mining', 1, data)
read_url('https://en.wikipedia.org/wiki/Data_mining', 2, data)

sorted_keys = sorted(data.keys())

with open("output.txt", "w", encoding="utf-8") as f:
    output_line = "Word".ljust(20) + "Frequency".ljust(15) + "URL_idx".ljust(15) + "\n"
    f.writelines(output_line)
    f.writelines('---------------------------------------------------------------------\n\n')
    for key in sorted_keys:
        output_string = str(key).ljust(20) + str(data[key][1]).ljust(15) + str(data[key][0]).ljust(15) + "\n"
        f.writelines(output_string)


In [7]:
# TODO
# Keyword tìm kiếm: python pickle
# Dùng pickle để lưu dictionary data xuống đĩa. Sau đó đọc lên và in ra 10 giá trị đầu tiên của nó.

# 1. Lưu data
import pickle
out = open("output.pickle", "wb")
pickle.dump(data, out)

# 2. Đọc lên và in ra
output = {}
with open("output.pickle", "rb") as f:
    output = pickle.load(f)

from itertools import islice
list(islice(output.items(), 10))

[('Web',
  [[1,
    48,
    80,
    84,
    94,
    98,
    103,
    125,
    128,
    138,
    141,
    144,
    154,
    162,
    165,
    168,
    184,
    187,
    202,
    211,
    213,
    229,
    243,
    318,
    512,
    533,
    537,
    552,
    555,
    571,
    631,
    693,
    710,
    766,
    787,
    793,
    810,
    836,
    878,
    895],
   41]),
 ('mining',
  [[1,
    49,
    77,
    81,
    86,
    96,
    100,
    105,
    126,
    129,
    140,
    143,
    146,
    155,
    160,
    164,
    167,
    170,
    186,
    189,
    215,
    245,
    285,
    303,
    320,
    453,
    487,
    514,
    535,
    539,
    549,
    554,
    557,
    560,
    621,
    633,
    694,
    712,
    739],
   90]),
 ('From', [[1], 3]),
 ('Wikipedia', [[1, 963], 4]),
 ('the',
  [[1,
    151,
    191,
    197,
    223,
    306,
    316,
    352,
    364,
    367,
    375,
    385,
    414,
    423,
    435,
    460,
    463,
    468,
    484,
    508,
    551,
    559,
    5